# Quantum Optimization with QAOA

In this notebook we will present:

1) How to prepare a state with a quantum circuit using the `QuantumCircuit` object from Qiskit.

2) How to build an observable using the `opflow` operators from Qiskit.

3) How to estimate the gain of our MaxCut instance using an observable and a quantum circuit.

4) An inspection of the QAOA variational quantum circuit.

5) How to perform classical optimization of the parameters on the QAOA circuit.

6) An analysis of the impact of layer repetition on the QAOA circuit.

## First we install and import dependencies

In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install matplotlib
!pip install pylatexenc
!pip install scipy

!git clone https://github.com/MarcoArmenta/qaoa_workshop.git

In [ ]:
import networkx as nx

from qiskit import QuantumCircuit
from qiskit.circuit.library.n_local.qaoa_ansatz import QAOAAnsatz
from qiskit.primitives import BackendEstimatorV2 as Estimator, BackendSamplerV2 as Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

from scipy.optimize import minimize

In [ ]:
import os
import sys
sys.path.insert(0, '/content/atelier_qaoa')

from QAOA_AlgoLab import *

In [ ]:
test_function()

## Quantum State Preparation

Let's start by preparing a quantum state with a quantum circuit. Let's prepare the state $|01011\rangle$.

In [ ]:
qc_trial = QuantumCircuit(5)
qc_trial.x([0,1,3])
qc_trial.draw('mpl')

We can verify that the prepared state is the correct state with the `qasm_simulator`.
The following code will print a dictionary of counts of the basis states obtained after running and measuring the quantum circuit `qc_trial` that we build before.

In [ ]:
# We first choose a backend to run the experiment. In this case, the qasm_simulator is used.
simulator = AerSimulator(shots=100)

# We copy the quantum circuit we build before to add measurements
qc_trial_state = qc_trial.copy()
# measure_all() methods builds a classical register on the QuantumCircuit
qc_trial.measure_all()

# We execute the experiment on the given backend with the corresponding specificacions
# and get the dictionary of counts of the experiment
counts = simulator.run(qc_trial).result().get_counts()
print(counts)

# Exercise 1
Build a quantum circuit to prepare the following state and then run it:

$ \frac{1}{\sqrt{2}} \big( |11101\rangle + |11001\rangle \big) $

In [ ]:
qc_ex_1 = QuantumCircuit(5)
# Write your code here


In [ ]:
exercise_superposition_state(qc_ex_1)

## Building the observable

We will use the `opflow` operators to build the observable. With the following code we build the first term of the gain operator corresponding to the orange and red nodes of the superhero graph.

In [ ]:
example_operator = SparsePauliOp(data=["IIIZZ"], coeffs=[-0.5])
print(example_operator)

## Exercise 2

Build the gain operator for the complete superhero graph.

In [ ]:
gain_operator_ex = #Write your code here

exercise_gain_operator(gain_operator_ex)

## Estimating the gain
Given a `QuantumCircuit` and an `opflow` operator we can estimate the gain using Qiskit.

In [ ]:
data = ["ZZIII", "IZZII", "ZIIZI", "IIZZI", "IIZIZ", "IIIZZ", "IIIII"]
coeffs = [-0.5,  -0.5, -0.5, -0.5, -0.5, -0.5, 0.5*6]

gain_operator = SparsePauliOp(data=data, coeffs=coeffs)

# If you want to know the details
# the function "eval_observable_on_state" is defined in the python file QAOA_AlgoLab.py
average_gain = eval_observable_on_state(gain_operator, qc_trial_state, simulator)

print(average_gain) #trial

## Exercise 3
Compute the gain for the state $ \frac{1}{\sqrt{2}} \big( |01011\rangle + |11011\rangle \big)$

In [ ]:
qc = QuantumCircuit(5)
# Build the circuit here

# uncoment the following line to draw the circuit
qc.draw('mpl')

In [ ]:
average_gain = eval_observable_on_state(gain_operator, qc, simulator)
exercise_average_gain(average_gain)

## Building the graph and showing solution states on it

We now display the graph with each superhero represented with a different color. Remember that the conexions on this graph represent shared superpowers.

In addition, we use two colors to color the interior of the nodes to represent the different teams: white (0) and gray (1), and we show here a particular choice, NOT the solution given by the algorithm. You can try to edit the variable `x` below to observe how the colors on the interior of the nodes change.

In [ ]:
# We build the graph using the object Graph from the networkx package
graph = nx.Graph()
# Adding the nodes
graph.add_nodes_from([0,1,2,3,4])
# Adding the edges
graph.add_edges_from([(0, 1), (0, 2), (1, 2), (1, 4), (2, 3), (3, 4)])

# Here, x is the bitstring representing the different teams with 0s for white and 1s for gray
# Try editing the bitstring to see how the teams coincide with the 0s and 1s
# Remember that the bitstring is read from right to left !!
x = '01011'
print_solution_graph(graph, x)

## Inspecting the QAOA quantum circuit

The Quantum Approximate Optimization Algorithm (QAOA) is a specific type of algorithm that prepares a parametrized quantum state. This circuit depends on the cost operator, which is the negative of the gain operator.

In [ ]:
data = ["ZZIII", "IZZII", "ZIIZI", "IIZZI", "IIZIZ", "IIIZZ", "IIIII"]
coeffs = [-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, 0.5*6]

gain_operator = SparsePauliOp(data=data, coeffs=coeffs)
cost_operator = - gain_operator

print(cost_operator)

The circuit of QAOA starts by putting all the qubits in an equal superposition to exploit quantum parallelism. Then, controlled RZ gates are applied accordingly with respect to the structure of the graph. The last step is a layer of RX gates.

Observe that the QAOAAnsatz object receives a variable called `reps`. This variables controls how many times the cost operator and the RX layer are repeated on the circuit of QAOA.

In [ ]:
qaoa_ansatz_1 = QAOAAnsatz(cost_operator, reps=1)

# Observe that the circuit is built accordingly to the cost_operator defined above
qaoa_ansatz_1.decompose().draw('mpl')

## Optimization of the parameters of the QAOA quantum circuit

In this part, we perform a classical optimization of the parameters on the quantum gates of the circuit presented above based on a specific classical optimizer. For this, we have to:

1) Choose an optimizer. We use SPSA.

2) Instantiate the QAOA algorithm from Qiskit.

3) Run the optimization process.

4) Get the state with higher counts.

In [ ]:
# Définir la fonction de coût classique à optimiser
def fonction_cout(
    params: list[complex], estimator: Estimator, circuit: QuantumCircuit, hamiltonien: SparsePauliOp
) -> float:
    """Fonction de cout qui calcule la valeur moyenne d'un observable (hamiltonien) pour un état donnée (circuit).
    Cette valeur moyenne représente le coût de la fonction de coût décrite par l'hamiltonien en entrée. Aussi, le
    circuit est paramétré et les paramètres sont définis dans le vecteurs params.
    Le tout est évalué à l'aide de l'estimateur (estimator).

    Args:
        params (list[complex]): Liste de paramètres à insérer dans le circuit en entrée
        estimator (Estimator): Calculateur utilisé pour estimer la valeur moyenne désirée.
        circuit (QuantumCircuit): Circuit paramétré de QAOA.
        hamiltonien (SparsePauliOp): Observable décrivant la fonction de coût du problème donné.

    Returns:
        float: Coût associé aux paramètres passés en entrée.
    """

    job = estimator.run([(circuit, hamiltonien, params)])
    cout = job.result()[0].data.evs
    return cout

In [ ]:
estimator = Estimator(backend=simulator)

# Initialisation des paramètres du circuit de QAOA à utiliser en premier lieu
params_init = np.zeros(qaoa_ansatz_1.num_parameters)

# Optimisation classique des paramètres du circuit de QAOA à l'aide de Scipy
res_opt = minimize(
    fonction_cout, params_init, args=(estimator, qaoa_ansatz_1.decompose(reps=2), cost_operator), method="COBYLA"
)  # , options={"tol": 1e-14}


# Extraction des informations suite à l'optimisation
cout_opt = res_opt.fun  # Cout optimal trouvé
params_opt = res_opt.x  # Paramètres optimaux trouvés

# Affichage des résultats obtenus
print("Cout optimal trouvé :", cout_opt)
print("Paramètres optimaux trouvés :", params_opt)

After the optimization process, each parameter on each gate has converged to a specific number. When these values are fixed we end up with a specific quantum circuit.

In [ ]:
qaoa_ansatz_1_opt = qaoa_ansatz_1.assign_parameters(params_opt)
qaoa_ansatz_1_opt.decompose().draw('mpl')

We now run this circuit using a `QuantumInstance` object from Qiskit and then show the histogram of probabilities.

In [ ]:
sampler = Sampler(backend=simulator)


optimal_qc_with_measurements = qaoa_ansatz_1_opt.copy()
optimal_qc_with_measurements.measure_all()
final_counts = samxpler.run([optimal_qc_with_measurements.decompose(reps=2)]).result()[0].data.meas.get_counts()
plot_histogram(final_counts, figsize=(12,4))

In [ ]:
# Get the state with maximum probability and print it
maximum_prob_state = max(final_counts, key=final_counts.get)
print("Basis state with highest probability: ", maximum_prob_state)

In [ ]:
# Show the obtained solution on the graph
print_solution_graph(graph, maximum_prob_state)

### Remark
Observe that in the example above the variable `reps = 1` on the instance of the QAOA algorithm. This is one of the reasons why the histogram doesn't show a strong difference between the actual solution of the MAXCUT problem and the other basis states. We will investigate what happens to this histogram when we increase the value of the variable `reps` in the next section.

## Exercise 4
What is the cut value of this solution?

In [ ]:
cut_value = # Put the cut value here

exercise_4(cut_value, maximum_prob_state)

## The impact of layer repetitions

We will now show 2 different histograms of basis states and probabilities obtained with different values of the variable `reps` on the QAOA object.

### `reps = 3`

In [ ]:
# Instantiate QAOA algorithm from Qiskit
qaoa_ansatz_3 = QAOAAnsatz(cost_operator, reps=3)

params_init = np.zeros(qaoa_ansatz_3.num_parameters)

# Optimisation classique des paramètres du circuit de QAOA à l'aide de Scipy
res_opt = minimize(
    fonction_cout, params_init, args=(estimator, qaoa_ansatz_3.decompose(reps=2), cost_operator), method="COBYLA"
)  # , options={"tol": 1e-14}


# Extraction des informations suite à l'optimisation
cout_opt = res_opt.fun  # Cout optimal trouvé
params_opt = res_opt.x  # Paramètres optimaux trouvés

qaoa_ansatz_3_opt = qaoa_ansatz_3.assign_parameters(params_opt)

optimal_qc_with_measurements = qaoa_ansatz_3_opt.copy()
optimal_qc_with_measurements.measure_all()
final_counts = sampler.run([optimal_qc_with_measurements.decompose(reps=2)]).result()[0].data.meas.get_counts()

# plot the histogram
plot_histogram(final_counts, figsize=(12,4))

In [ ]:
maximum_prob_state = max(final_counts, key=final_counts.get)
print_solution_graph(graph, maximum_prob_state)

### `reps = 8`

In [ ]:
# Instantiate QAOA algorithm from Qiskit
qaoa_ansatz_8 = QAOAAnsatz(cost_operator, reps=8)

params_init = np.zeros(qaoa_ansatz_8.num_parameters)

# Optimization process
res_opt = minimize(
    fonction_cout, params_init, args=(estimator, qaoa_ansatz_8.decompose(reps=2), cost_operator), method="COBYLA"
)  # , options={"tol": 1e-14}


# Extraction des informations suite à l'optimisation
cout_opt = res_opt.fun  # Cout optimal trouvé
params_opt = res_opt.x  # Paramètres optimaux trouvés

# Getting the histogram of counts of the optimal circuit
qaoa_ansatz_8_opt = qaoa_ansatz_8.assign_parameters(params_opt)

optimal_qc_with_measurements = qaoa_ansatz_8_opt.copy()
optimal_qc_with_measurements.measure_all()
final_counts = sampler.run([optimal_qc_with_measurements.decompose(reps=2)]).result()[0].data.meas.get_counts()

# plot the histogram
plot_histogram(final_counts, figsize=(12,4))

In [ ]:
maximum_prob_state = max(final_counts, key=final_counts.get)
print_solution_graph(graph, maximum_prob_state)

## Observations

You may have noticed that QAOA doesn't always obtains the optimal solution and that this sometimes depends on how high is the value of the `reps` variable. Remember that QAOA stands for Quantum <strong><i>Approximate</i></strong> Optimization Algorithm, which means that the algorithm may not always find the optimal solution. Nevertheless, QAOA is still usefull to obtain approximate solutions to hard problems that cannot be obtained by classical algorithms.

Finally, we mention a mathematical result related to the phenomenon of getting better solutions when we increase the variable `reps`.

### Theorem.
The quality of the approximation made by the algorithm increases with the number of layers.


## Without optimization

According to the theory of quantum annealing, one can classicaly compute a set of parameters that allow us to
compute the average value that is closest to an optimal result. It is not clear if this approach
will also work on systems with a bigger systems.


In [ ]:
reps = 3

# qaoa_n_reps = QAOA(optimizer=optimizer,quantum_instance=quantum_instance,reps=reps)

betas = [-1., -0.6, -0.2]
gammas = [0.2, 0.6, 1.]

initial_qc = qaoa_ansatz_3.assign_parameters(betas + gammas)
initial_qc.decompose(reps=2).draw()

In [ ]:
initial_qc.measure_all()
final_counts = sampler.run([initial_qc.decompose(reps=2)]).result()[0].data.meas.get_counts()
plot_histogram(final_counts, figsize=(12,4))

## Appreciation and evaluation form

You can use this notebook to help you solve the test [HERE](https://forms.office.com/Pages/ResponsePage.aspx?id=fn_X3Rof9E-j8Sm0oIYh58RoI-LCaGxAqo7x_1-ZdwRUOFdSRkNXVDNVTEFIWjVFODJLNDA4QUQ0OSQlQCN0PWcu).
You have to respond correctly to at least 2 questions out of 3 on the test inside the form.
There is also a section concerning your appreciation of the workshop. You have 24 hours to respond!



# Link to the form

[THE FORM](https://forms.office.com/Pages/ResponsePage.aspx?id=fn_X3Rof9E-j8Sm0oIYh58RoI-LCaGxAqo7x_1-ZdwRUOFdSRkNXVDNVTEFIWjVFODJLNDA4QUQ0OSQlQCN0PWcu)